# Model sec2sec z użyciem wyrazów

## Stałe

In [3]:
batch_size = 64  
epochs = 50  
latent_dim = 256  
embed_dim = 32
num_samples = 10000
data_path = './data/pol-eng-short.txt'
clean_data_path = './data/clean-pol-eng-short.txt'

## Czyszczenie tekstu

In [4]:
import string

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read()
    table = str.maketrans('', '', string.punctuation)
    clean_lines = lines.lower().translate(table)

with open(clean_data_path, 'w', encoding='utf-8') as f:
    f.write(clean_lines)

## Wektoryzacja

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

with open(clean_data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: num_samples]:
    input_text, target_text = line.split('\t')
    #input_text = '<s> ' + input_text + ' <e>'
    target_text = "<s> " + target_text + " <e>"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text.split(' '):
        if word not in input_words:
            input_words.add(word)
    for word in target_text.split(' '):
        if word not in target_words:
            target_words.add(word)
            
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt.split(' ')) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split(' ')) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 3221
Number of unique output tokens: 6246
Max sequence length for inputs: 7
Max sequence length for outputs: 11


## Generacja danych

In [6]:
import numpy as np

# slowniki slow
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

# wypelnienie wejsc zerami
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for text_index, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for word_index, (word) in enumerate(input_text.split(' ')):
        encoder_input_data[text_index, word_index, input_token_index[word]] = 1.
    for word_index, (word) in enumerate(target_text.split(' ')):
        decoder_input_data[text_index, word_index, target_token_index[word]] = 1.
        if word_index > 0:
            decoder_target_data[text_index, word_index-1, target_token_index[word]] = 1.

## Model treningowy

In [20]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model\
model.save('s2s-word-inz.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
  64/8000 [..............................] - ETA: 7:46 - loss: 3.2525

KeyboardInterrupt: 

## Model produkcyjny

In [12]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense

model = load_model('s2s-word-inz.h5')

for layer in model.layers:
    print(layer.get_config())
    print('\n')
    
#inputs
encoder_inputs = Input(shape=(None, num_encoder_tokens))
decoder_inputs = Input(shape=(None, num_decoder_tokens))

#inference encoder
encoder_lstm = model.get_layer(index = 2)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

#inference decoder
decoder_lstm = model.get_layer(index = 3)
decoder_dense = model.get_layer(index = 4)
    
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict(
#    (i, char) for char, i in input_token_index.items())
reverse_input_word_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_token_index.items())



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["<s>"]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += sampled_word + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == "<e>" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

{'name': 'input_1', 'sparse': False, 'batch_input_shape': (None, None, 3221), 'dtype': 'float32'}


{'name': 'input_2', 'sparse': False, 'batch_input_shape': (None, None, 6246), 'dtype': 'float32'}


{'activity_regularizer': None, 'use_bias': True, 'kernel_constraint': None, 'go_backwards': False, 'name': 'lstm_1', 'unit_forget_bias': True, 'bias_regularizer': None, 'recurrent_dropout': 0.0, 'recurrent_activation': 'hard_sigmoid', 'bias_initializer': {'config': {}, 'class_name': 'Zeros'}, 'activation': 'tanh', 'return_state': True, 'dropout': 0.0, 'bias_constraint': None, 'implementation': 1, 'kernel_regularizer': None, 'unroll': False, 'return_sequences': False, 'trainable': True, 'recurrent_initializer': {'config': {'seed': None, 'gain': 1.0}, 'class_name': 'Orthogonal'}, 'units': 256, 'recurrent_regularizer': None, 'stateful': False, 'recurrent_constraint': None, 'kernel_initializer': {'config': {'distribution': 'uniform', 'mode': 'fan_avg', 'seed': None, 'scale': 1.0}, 'class_name'